In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import lagrange
import matplotlib.pyplot as plt

## 二：内容和方法
### 1.插补用户用电量数据缺失值。
用户用电量数据呈现一定的周期性关系，missing_data.csv表中存放了用户A、B和C的用电量数据，其中存在缺失值，需要进行缺失值插补才能进行下一步分析。现在需要完成以下步骤：<br>
（1）读取missing_data.csv表的数据并保存为pandas数据框。<br>
（2）查询缺失值所在位置。<br>
（3）使用SciPy库中interpolate模块中的lagrange对数据进行拉格朗日插值。<br>
（4）检查数据中是否存在缺失值。<br>

In [2]:
# (1)
data1 = pd.read_excel('D:/root/Desktop/各科/kk/实验5_Pandas数据预处理/missing_data.csv.xls',encoding='utf8',header=None)
data1.columns = ['A','B','C']
data1

,A,B,C
0,235.8333,324.0343,478.3231
1,236.2708,325.6379,515.4564
2,238.0521,328.0897,517.0909
3,235.9063,NaN,514.8900
4,236.7604,268.8324,NaN
5,NaN,404.0480,486.0912
6,237.4167,391.2652,516.2330
7,238.6563,380.8241,NaN
8,237.6042,388.0230,435.3508
9,238.0313,206.4349,487.6750


In [3]:
# (2)
index = np.where(data1[data1.columns[:]].isna())
[[i,data1.columns[j]] for i,j in zip(index[0],index[1])]

[[3, 'B'],
 [4, 'C'],
 [5, 'A'],
 [7, 'C'],
 [10, 'B'],
 [10, 'C'],
 [12, 'A'],
 [16, 'C'],
 [19, 'B'],
 [20, 'B']]

In [4]:
# (3)

A1 = index[0][np.where(index[1]==0)]
A = data1['A'].dropna()
f_lag_A = lagrange(A.index,list(A))
Ay = f_lag_A(A1)
data1['A'][A1] = Ay
print(A1,Ay)

B1 = index[0][np.where(index[1]==1)]
B = data1['B'].dropna()
f_lag_B = lagrange(B.index,list(B))
By = f_lag_B(B1)
data1['B'][B1]=By
print(B1,By)

C1 = index[0][np.where(index[1]==2)]
C = data1['C'].dropna()
f_lag_C = lagrange(C.index,list(C))
Cy = f_lag_C(C1)
data1['C'][C1]=Cy
print(C1,Cy)

[ 5 12] [224.82362466 236.86154825]
[ 3 10 19 20] [ 1.61971831e+03  2.17555992e+02 -7.94294127e+05 -1.20871392e+07]
[ 4  7 10 16] [519.13106536 491.79295169 613.42630189 622.27836034]


In [5]:
# (4)
np.any(data1.isna())

False

### 2.标准化建模专家样本数据。

（1）读取model.csv数据并保存为pandas数据框。<br>
（2）定义标准差标准化函数。<br>
（3）使用函数分别对3列数据进行标准化。<br>
（4）查看标准化后的数据。<br>

In [6]:
# (1)
data2 = pd.read_excel('D:/root/Desktop/各科/kk/实验5_Pandas数据预处理/5_models.xls')
data2

,电量趋势下降指标,线损指标,告警类指标,是否窃漏电
0,4,1,1,1
1,4,0,4,1
2,2,1,1,1
3,9,0,0,0
4,3,1,0,0
...,...,...,...,...
286,4,1,2,0
287,1,0,2,0
288,5,1,2,1
289,2,1,0,0


In [7]:
# (2)
def StandardScaler(data):
    data=(data-data.mean())/data.std()
    return data

In [8]:
# (3)
data2[['电量趋势下降指标','线损指标','告警类指标']].agg(StandardScaler)

,电量趋势下降指标,线损指标,告警类指标
0,0.612777,0.974549,-0.161216
1,0.612777,-1.022590,2.653616
2,-0.343352,0.974549,-0.161216
3,3.003099,-1.022590,-1.099494
4,0.134712,0.974549,-1.099494
...,...,...,...
286,0.612777,0.974549,0.777061
287,-0.821417,-1.022590,0.777061
288,1.090841,0.974549,0.777061
289,-0.343352,0.974549,-1.099494


### 3.招聘数据探索分析。

（1）读取上述job_inf.csv数据并存为一个名叫job_info的数据框。<br>
（2）将列命名为：['公司', '岗位', '工作地点', '工资', '发布日期']。<br>
（3）哪个岗位招聘需求最多？<br>
（4）取出9月3日发布的招聘信息。<br>
（5）找出工作地点在深圳的数据分析师招聘信息。<br>
（6）取出每条招聘信息中的最低工资与最高工资，单位为“元/月”，若招聘信息中无工资数据则无需处理。（如2-2.5万/月，则最低工资为20000，最高工资为25000。）

In [9]:
# (1)
job_info = pd.read_excel('D:/root/Desktop/各科/kk/实验5_Pandas数据预处理/job_inf.csv.xlsm',header=None)
job_info

,0,1,2,3,4
0,字节跳动,数据产品经理,北京,2-3.5万/月,44077
1,甲骨文（中国）软件系统有限公司,数据产品经理,长沙,NaN,44077
2,莱茵技术(上海)有限公司 TUV Rhei...,数据产品经理,上海-静安区,NaN,44077
3,百度在线网络技术（北京）有限公司...,数据产品经理,北京,2-4万/月,44077
4,携程旅行网业务区,数据产品经理,上海-长宁区,1.5-2万/月,44077
...,...,...,...,...,...
5495,深圳市创荣发电子有限公司,仓库数据分析员,深圳,2-2.5万/月,44078
5496,深圳市宝华诚投资管理有限公司,仓库数据分析员,深圳-龙华新区,6-8千/月,44078
5497,广州市卡思投资有限公司,仓库数据分析员,深圳-福田区,0.8-1万/月,44078
5498,深圳市迪比科电子科技有限公司,仓库数据分析员,深圳-龙华新区,1.5-2.5万/月,44078


In [10]:
# (2)
job_info.columns = ['公司', '岗位', '工作地点', '工资', '发布日期']
job_info.columns

Index(['公司', '岗位', '工作地点', '工资', '发布日期'], dtype='object')

In [11]:
# (3)
vc = job_info['岗位'].value_counts()
vc.index[np.where(vc.values.max())][0],vc.values.max()

('数据分析师', 4000)

In [12]:
# (4)
job_info['发布日期'] = pd.DatetimeIndex(job_info['发布日期'])
[i for i in job_info.values if i[4].month == 9 and i[4].day == 3]
# job_info.loc[job_info['发布日期'].month=9,:]
# job_info['发布日期'].month

[]

In [13]:
# (5)
job_info[job_info['工作地点']=='深圳'][job_info['岗位']=='数据分析师']

D:\root\software\anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,公司,岗位,工作地点,工资,发布日期
1016,瑞庭网络技术（上海）有限公司深圳...,数据分析师,深圳,7-9千/月,1970-01-01 00:00:00.000044077
1070,广州奕晟网络科技有限公司,数据分析师,深圳,6-8千/月,1970-01-01 00:00:00.000044077
1107,鹏元征信有限公司,数据分析师,深圳,0.7-1.2万/月,1970-01-01 00:00:00.000044077
1196,深圳诺德安达双语学校,数据分析师,深圳,1-1.5万/月,1970-01-01 00:00:00.000044077
1203,丽晶维珍妮内衣（深圳）有限公司,数据分析师,深圳,5-8千/月,1970-01-01 00:00:00.000044077
...,...,...,...,...,...
4722,雅特生科技,数据分析师,深圳,6-8千/月,1970-01-01 00:00:00.000044077
4775,深圳市加力邦德咨询有限公司,数据分析师,深圳,1.5-2万/月,1970-01-01 00:00:00.000044077
4797,深圳市欧烨信息技术有限公司,数据分析师,深圳,4.5-6千/月,1970-01-01 00:00:00.000044077
4868,太平船务（中国）有限公司,数据分析师,深圳,4.5-5千/月,1970-01-01 00:00:00.000044077


In [14]:
# (6)
imfor = [(j,[float(k.replace('元','').replace('千以',''))*10000 for k in i[0:-3].split('-')]) for j,i in zip(job_info['公司'],job_info['工资']) if type(i) == str]
[(i[0], '最低为：'+str(i[1][0]),'最高为：'+str(i[1][-1])) for i in imfor]

[('字节跳动', '最低为：20000.0', '最高为：35000.0'),
 ('百度在线网络技术（北京）有限公司...', '最低为：20000.0', '最高为：40000.0'),
 ('携程旅行网业务区', '最低为：15000.0', '最高为：20000.0'),
 ('杭州默沙东制药有限公司', '最低为：8000.0', '最高为：15000.0'),
 ('卡特彼勒(苏州)物流有限公司', '最低为：45000.0', '最高为：60000.0'),
 ('IBM China', '最低为：20000.0', '最高为：40000.0'),
 ('辉瑞（中国）研究开发有限公司', '最低为：15000.0', '最高为：20000.0'),
 ('浙江华为通信技术有限公司', '最低为：10000.0', '最高为：15000.0'),
 ('太平洋保险在线服务科技有限公司深...', '最低为：45000.0', '最高为：60000.0'),
 ('上海晨光文具股份有限公司', '最低为：8000.0', '最高为：10000.0'),
 ('深圳市大疆创新科技有限公司', '最低为：10000.0', '最高为：15000.0'),
 ('苏宁易购集团股份有限公司', '最低为：15000.0', '最高为：20000.0'),
 ('上海华夏邓白氏商业信息咨询有限公...', '最低为：35000.0', '最高为：45000.0'),
 ('广州立白企业集团有限公司', '最低为：300000.0', '最高为：500000.0'),
 ('金蝶软件（中国）有限公司', '最低为：10000.0', '最高为：20000.0'),
 ('传智播客教育集团深圳分校', '最低为：25000.0', '最高为：30000.0'),
 ('大华银行（中国）有限公司', '最低为：30000.0', '最高为：40000.0'),
 ('优信集团', '最低为：10000.0', '最高为：13000.0'),
 ('北京达佳互联信息技术有限公司', '最低为：8000.0', '最高为：10000.0'),
 ('新华三技术有限公司', '最低为：10000.0', '最高为：20000.0'),
 ('广州汇骏信息咨询服务有限公司', 